<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/01_Qiskit_%ED%9A%8C%EB%A1%9C%EC%9D%98%EA%B8%B0%EC%B4%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Qiskit 패캐지 설치

In [ ]:
!python3 -m pip install qiskit[visualization]

패키지 불러오기

In [ ]:
import numpy as np
from qiskit import QuantumCircuit

양자회로 만들기

In [ ]:
# Create a Quantum Circuit. acting on a quantum register of three qubits
circ = QuantumCircuit(3)

양자충첩(H gate)과 얽힘(CX gate)만들기

In [ ]:
#ADD a Hadamard gate on qubit 0, putting this qubit in superposition.
circ.h(0)
#Add a CX(CNOT, Controlled NOT) gate on control qubit 0 and target qubit 1, putting the qubits in a Bell state.
circ.cx(0, 1)
#Add a CX(CNOT) gate on control qubit 0 and target qubit 2, putting the qubits in a GHZ state.
circ.cx(0, 2)

회로 시각화하기

In [ ]:
circ.draw('mpl') ##Matplot Library로 시각화

다중 큐비트 시스템의 상태를 나타낼 때, Qiskit은 대부분의 물리 교재에서 사용되는 텐서(tensor)의 순서와는 반대 되는 순서를 사용한다. 
 개의 큐비트가 있고, 큐비트 
 는 
 로 표시된다고 가정하자. Qiskit에서는 (기저 벡터들을 
 로 표현하기 위하여) 
 큐비트를 텐서 곱의 가장 왼편에 위치하도록 만드는 순서체계(ordering)를 사용한다.

예를 들어, 큐비트 0이 0 상태에 있고, 큐비트 1이 0 상태, 큐비트 2가 1 상태에 있을 때, Qiskit은 이 상태를 
 로 표현하는데, 대부분의 물리 교과서에서는 
 로 표현한다.

이러한 표현의 차이는 다중 큐비트 연산이 행렬로 표현될 때 나타나게 된다. 예를 들어 큐비트 0이 제어 큐비트이고 큐비트 1이 목표 큐비트일 때, controlled-X (
) 연산은 Qiskit에서 다음과 같이 표현된다.

 
 


회로 시뮬레이션하기

In [ ]:
from qiskit.quantum_info import Statevector

# Set the intial state of the simulator to the ground state using from_int
state = Statevector.from_int(0, 2**3)

# Evolve the state by the quantum circuit
state = state.evolve(circ)

#draw using latex
state.draw('latex')

In [ ]:
from qiskit.visualization import array_to_latex

#Alternative way of representing in latex
array_to_latex(state)

In [ ]:
#큐시피어로 표시
state.draw('qsphere')

In [ ]:
#힌튼 다이어그램
state.draw('hinton')

회로의 유니터리 표현

In [ ]:
from qiskit.quantum_info import Operator

U = Operator(circ)

# Show the results
U.data

**OpenQASM 백엔드**
시뮬레이터들은 회로의 행렬 상태를 보여주고 이상적인 회로의 상태 결과에 대한 정보를 주기 때문에 유용하다. 그러나 실제의 실험은 각각의 큐비트(보통 computational 
 기저로 표현된다)를 측정 함으로써 끝난다. 측정이 없다면, 상태에 대한 정보를 얻을 수 없다. 또한 측정을 통해 양자 시스템은 고전적인 비트로 붕괴하게 된다.

In [ ]:
# Create a Quantum Circuit
meas = QuantumCircuit(3, 3)
meas.barrier(range(3))
# map the quantum measurement to the classical bits
meas.measure(range(3), range(3))

# The Qiskit circuit object supports composition.
# Here the meas has to be firest and front=True (putting it before)
# as compose must put a smaller circuit into a larger one
qc = meas.compose(circ, range(3), front=True)

#drawing the circuit
qc.draw('mpl')


In [ ]:
# Adding the transpiler to reduce the circuit to QASM instructions
# supported by the backend
from qiskit import transpile

# User AerSimulator
from qiskit_aer import AerSimulator

backend = AerSimulator()

# First we have to transpile the quantum circuit
# to the low-level QASM instructions used by the backend
qc_compiled = transpile(qc, backend)

# Execute the circuit on the qasm simulator. We've set the number of repeates of the circuit 
# to be 1,024, which is the default.
job_sim = backend.run(qc_compiled, shots=1024)

# Grab the results from the job
result_sim = job_sim.result()

In [ ]:
counts = result_sim.get_counts(qc_compiled)
print(counts)

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright